In [8]:
from FusekiCommunicator import FusekiCommunicatior
import numpy as np
import matplotlib.pyplot as plt
from typing import Any, Dict
from pydantic import BaseModel
from rdflib import Graph, URIRef, Literal

class ResponseHead(BaseModel):
    vars: list[str]

class ResponseResults(BaseModel):
    bindings: list[Dict[str, Any]]

class SearchResponse(BaseModel):
    head: ResponseHead
    results: ResponseResults
    
from starlette.responses import RedirectResponse
from starlette.status import (
    HTTP_303_SEE_OTHER,
    HTTP_400_BAD_REQUEST,
    HTTP_500_INTERNAL_SERVER_ERROR,
)
from fastapi import APIRouter, HTTPException
from loguru import logger

from datetime import datetime, timedelta


In [134]:
fuseki_jena_url = "192.168.4.26"
fuseki_jena_port = "30731"
fuseki_jena_dataset_name = "slice"
fuseki = FusekiCommunicatior(
    fuseki_jena_url, fuseki_jena_port, fuseki_jena_dataset_name
)

http://127.0.0.1:8000/slice


In [9]:
# def delete_old_graphs(fuseki):

#     try:
#         # Calculate the timestamp for 24 hours ago
#         now = datetime.utcnow()
#         cutoff_time = now - timedelta(days=1)
#         cutoff_timestamp = cutoff_time.strftime("%Y-%m-%dT%H:%M:%SZ")

#         # SPARQL update query to delete named graphs older than cutoff_timestamp
#         delete_query = f"""
#             PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
#             DELETE WHERE {{
#               GRAPH ?g {{
#                 ?s ?p ?o .
#               }}
#               FILTER (str(?g) < "{cutoff_timestamp}"^^xsd:dateTime)
#             }}
#             """

#         # Execute the SPARQL update
#         fuseki.query()

#         print(f"Deleted graphs older than {cutoff_timestamp}")

#     except Exception as e:
#         print(f"An error occurred during cleanup: {e}")

In [28]:
import requests
from datetime import datetime, timedelta


def delete_old_graphs(fuseki, cutoff_timestamp):
    """
    Deletes graphs in Apache Jena with IDs (graph URIs) before the given Unix timestamp.

    Args:
    - base_url: The base URL of the Apache Jena server (e.g., 'http://localhost:3030/dataset').
    - cutoff_timestamp: The Unix timestamp before which graphs should be deleted.
    """
    # Construct the SPARQL update query
    
    sparql_update_query = f"""
    DELETE WHERE {{
        GRAPH ?g {{
            ?s ?p ?o
        }}
        FILTER (xsd:integer(STRAFTER(STR(?g), 'timestamp:')) < {cutoff_timestamp})
    }}
    """
    


    print(sparql_update_query)
    fuseki.update_query(sparql_update_query)





In [29]:
cutoff_time = datetime.now() - timedelta(days=1)
cutoff_timestamp = int(cutoff_time.timestamp())
delete_old_graphs(fuseki, cutoff_timestamp)


    DELETE WHERE {
        GRAPH ?g {
            ?s ?p ?o
        }
        FILTER (xsd:integer(STRAFTER(STR(?g), 'timestamp:')) < 1730911494)
    }
    
QueryBadFormed: A bad request has been sent to the endpoint: probably the SPARQL query is badly formed. 

Response:
b'Encountered " "filter" "FILTER "" at line 6, column 9.\nWas expecting one of:\n    <IRIref> ...\n    <PNAME_NS> ...\n    <PNAME_LN> ...\n    <BLANK_NODE_LABEL> ...\n    <VAR1> ...\n    <VAR2> ...\n    "graph" ...\n    "true" ...\n    "false" ...\n    <INTEGER> ...\n    <DECIMAL> ...\n    <DOUBLE> ...\n    <INTEGER_POSITIVE> ...\n    <DECIMAL_POSITIVE> ...\n    <DOUBLE_POSITIVE> ...\n    <INTEGER_NEGATIVE> ...\n    <DECIMAL_NEGATIVE> ...\n    <DOUBLE_NEGATIVE> ...\n    <STRING_LITERAL1> ...\n    <STRING_LITERAL2> ...\n    <STRING_LITERAL_LONG1> ...\n    <STRING_LITERAL_LONG2> ...\n    "(" ...\n    <NIL> ...\n    "}" ...\n    "[" ...\n    <ANON> ...\n    "." ...\n    "<<" ...\n    \n'


In [16]:
cutoff_timestamp

1730911175

In [ ]:

# if __name__ == "__main__":
#     # Base URL of the Apache Jena server
#     base_url = 'http://localhost:3030/dataset'

#     # Define the cutoff time (e.g., 1 day ago)
#     cutoff_time = datetime.now() - timedelta(days=1)
#     cutoff_timestamp = int(cutoff_time.timestamp())

#     # Delete old graphs
#     delete_old_graphs(base_url, cutoff_timestamp)

In [34]:
# print(cutoff_timestamp)

# sparql_update_query = f"""
# DELETE WHERE {{
#     GRAPH ?g {{
#         ?s ?p ?o
#     }}
#     FILTER (xsd:integer(STRAFTER(STR(?g), 'timestamp:')) < {cutoff_timestamp})
# }}
# """

fuseki.update_query("DROP GRAPH <timestamp:1730279403441>")


In [37]:
def read_file(fname):
    with open(fname, "r") as f:
        return f.read()

def perform_query(query, fuseki):
    valid, msg = fuseki.validate_sparql(query, "query")
    if valid:
        result = fuseki.read_query(query)
        bindings = []
        for item in result.bindings:
            new_item: Dict[str, Any] = {}
            for key in item:
                new_item[key] = {}
                for property in item[key].__dict__:
                    if (
                        property != "variable"
                        and item[key].__dict__[property] is not None
                    ):
                        new_item[key][property] = item[key].__dict__[property]
                        if property == "lang":
                            new_item[key]["xml:lang"] = new_item[key].pop("lang")
            bindings.append(new_item)
        return SearchResponse(
                head=ResponseHead(vars=result.head["vars"]),
                results=ResponseResults(bindings=bindings),
            )
    print(msg)
    return None

In [135]:
timestamps = np.array(
    [
        timestamp["graphURI"]["value"]
        for timestamp in perform_query(
            read_file("query_timestamps.txt"), fuseki
        ).results.bindings
    ]
)

for ts in timestamps:
    print(ts)

EndPointNotFound: It was not possible to connect to the given endpoint: check it is correct. 

Response:
b'{"detail":"Not Found"}'


AttributeError: 'NoneType' object has no attribute 'bindings'

In [64]:
def drop_old_graphs(fuseki):
    timestamps = np.array(
        [
            timestamp["graphURI"]["value"]
            for timestamp in perform_query(
                read_file("query_timestamps.txt"), fuseki
            ).results.bindings
        ]
    )
    print(f"{timestamps.size} timestamps retreived.")
    # Define the cutoff time (e.g., 1 day ago)
    cutoff_time = datetime.now() - timedelta(days=1)
    cutoff_timestamp = int(cutoff_time.timestamp())*1000

    for i in range(len(timestamps)):
        ts = int(timestamps[i].split(':')[-1])
        
        if ts<cutoff_timestamp:
            fuseki.update_query("DROP GRAPH <" + timestamps[i]+'>')
            print("dropped "+timestamps[i])


In [66]:
drop_old_graphs(fuseki)

3 timestamps retreived.


In [107]:
from urllib.parse import urlencode

import requests
def local_query(query: str | None = None) -> Any:
    """
    Queries Local Metadata service
    """
    
    if query == None:
        query=query

    params = {"query": query}
    encoded_query = urlencode(params)
    base_url = "http://192.168.4.26:32732/api/v0/graph"
    full_url = f"{base_url}?{encoded_query}"

    print("full_url =", full_url)

    response = requests.get(full_url)

    return response.json()

In [125]:
def local_update_query(query: str | None = None) -> Any:
    """
    Queries Local Metadata service
    """
    
    if query == None:
        query=query

    params = {"body": query}
    encoded_query = urlencode(params)
    base_url = "http://192.168.4.26:32732/api/v0/graph"
    full_url = f"{base_url}?{encoded_query}"

    print("full_url =", full_url)

    response = requests.patch(full_url)

    return response.json()

In [110]:
query=read_file("query_timestamps.txt")
print(query)

PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?graphURI WHERE {
  GRAPH ?graphURI {}
  FILTER regex(str(?graphURI), "^timestamp:")
}
ORDER BY ASC(xsd:integer(replace(str(?graphURI), "^timestamp:", "")))
# LIMIT 100



In [112]:
local_query(query)

full_url = http://192.168.4.26:32732/api/v0/graph?query=PREFIX+xsd%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23%3E%0A%0ASELECT+%3FgraphURI+WHERE+%7B%0A++GRAPH+%3FgraphURI+%7B%7D%0A++FILTER+regex%28str%28%3FgraphURI%29%2C+%22%5Etimestamp%3A%22%29%0A%7D%0AORDER+BY+ASC%28xsd%3Ainteger%28replace%28str%28%3FgraphURI%29%2C+%22%5Etimestamp%3A%22%2C+%22%22%29%29%29%0A%23+LIMIT+100%0A


{'head': {'vars': ['graphURI']},
 'results': {'bindings': [{'graphURI': {'value': 'timestamp:1730989204949',
     'type': 'uri'}},
   {'graphURI': {'value': 'timestamp:1731019205919', 'type': 'uri'}},
   {'graphURI': {'value': 'timestamp:1731049205972', 'type': 'uri'}}]}}

In [113]:
timestamps = np.array(
    [
        timestamp["graphURI"]["value"]
        for timestamp in local_query(
            read_file("query_timestamps.txt")
        )['results']['bindings']
    ]
)

full_url = http://192.168.4.26:32732/api/v0/graph?query=PREFIX+xsd%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23%3E%0A%0ASELECT+%3FgraphURI+WHERE+%7B%0A++GRAPH+%3FgraphURI+%7B%7D%0A++FILTER+regex%28str%28%3FgraphURI%29%2C+%22%5Etimestamp%3A%22%29%0A%7D%0AORDER+BY+ASC%28xsd%3Ainteger%28replace%28str%28%3FgraphURI%29%2C+%22%5Etimestamp%3A%22%2C+%22%22%29%29%29%0A%23+LIMIT+100%0A


In [114]:
len(timestamps)

3

In [126]:
def clear_data():

    timestamps = np.array(
        [
            timestamp["graphURI"]["value"]
            for timestamp in local_query(
                read_file("query_timestamps.txt")
            )['results']['bindings']
        ]
    )
    
    cutoff_time = datetime.now() - timedelta(days=0)
    cutoff_timestamp = int(cutoff_time.timestamp())*1000

    for i in range(len(timestamps)):
        ts = int(timestamps[i].split(':')[-1])
        
        if ts<cutoff_timestamp:
            res=local_update_query("DROP GRAPH <" + timestamps[i]+'>')
            print(res)
            # print("dropped "+timestamps[i])

In [127]:
clear_data()

full_url = http://192.168.4.26:32732/api/v0/graph?query=PREFIX+xsd%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23%3E%0A%0ASELECT+%3FgraphURI+WHERE+%7B%0A++GRAPH+%3FgraphURI+%7B%7D%0A++FILTER+regex%28str%28%3FgraphURI%29%2C+%22%5Etimestamp%3A%22%29%0A%7D%0AORDER+BY+ASC%28xsd%3Ainteger%28replace%28str%28%3FgraphURI%29%2C+%22%5Etimestamp%3A%22%2C+%22%22%29%29%29%0A%23+LIMIT+100%0A
full_url = http://192.168.4.26:32732/api/v0/graph?body=DROP+GRAPH+%3Ctimestamp%3A1730989204949%3E
{'detail': [{'type': 'missing', 'loc': ['body'], 'msg': 'Field required', 'input': None, 'url': 'https://errors.pydantic.dev/2.6/v/missing'}]}
full_url = http://192.168.4.26:32732/api/v0/graph?body=DROP+GRAPH+%3Ctimestamp%3A1731019205919%3E
{'detail': [{'type': 'missing', 'loc': ['body'], 'msg': 'Field required', 'input': None, 'url': 'https://errors.pydantic.dev/2.6/v/missing'}]}
full_url = http://192.168.4.26:32732/api/v0/graph?body=DROP+GRAPH+%3Ctimestamp%3A1731049205972%3E
{'detail': [{'type': 'missing', 'l

In [132]:
sparql_endpoint = "http://192.168.4.26:32732/api/v0/graph"

sparql_update_query = "DROP GRAPH <" + timestamps[0]+">"

response = requests.patch(sparql_endpoint, data={'update': sparql_update_query},
                             headers={'Content-Type': 'application/x-www-form-urlencoded'})

In [133]:
response

<Response [422]>

In [128]:
headers = {'Content-Type': 'application/x-www-form-urlencoded'}
response = requests.post(sparql_endpoint, data=sparql_update_query, headers=headers)

In [129]:
response

<Response [405]>

In [138]:
from urllib.parse import urlencode

import requests
def super_local_query(query: str | None = None) -> Any:
    """
    Queries Local Metadata service
    """
    
    if query == None:
        query=query

    params = {"query": query}
    encoded_query = urlencode(params)
    base_url = "http://127.0.0.1:8000/api/v0/graph"
    full_url = f"{base_url}?{encoded_query}"

    print("full_url =", full_url)

    response = requests.get(full_url)

    return response.json()

In [139]:
timestamps = np.array(
    [
        timestamp["graphURI"]["value"]
        for timestamp in super_local_query(
            read_file("query_timestamps.txt")
        )['results']['bindings']
    ]
)

full_url = http://127.0.0.1:8000/api/v0/graph?query=PREFIX+xsd%3A+%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23%3E%0A%0ASELECT+%3FgraphURI+WHERE+%7B%0A++GRAPH+%3FgraphURI+%7B%7D%0A++FILTER+regex%28str%28%3FgraphURI%29%2C+%22%5Etimestamp%3A%22%29%0A%7D%0AORDER+BY+ASC%28xsd%3Ainteger%28replace%28str%28%3FgraphURI%29%2C+%22%5Etimestamp%3A%22%2C+%22%22%29%29%29%0A%23+LIMIT+100%0A


In [140]:
print(timestamps)

['timestamp:1730989204949' 'timestamp:1731019205919'
 'timestamp:1731049205972']
